In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-09 21:21:17--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2025-06-09 21:21:17 (106 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# Read the text file
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

print(f"Length of dataset in characters: {len(text)}")
print(text[:500])  # Preview the first 500 characters

Length of dataset in characters: 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [3]:
unique_chars = sorted(set(text))
indexes = list(range(len(unique_chars)))
char_to_int = dict(zip(unique_chars, indexes))
int_to_char = dict(zip(indexes, unique_chars))

In [4]:
def encode(x):
  return [char_to_int[c] for c in x]

def decode(x):
  return ''.join([int_to_char[i] for i in x])

In [5]:
encoded_text = encode(text)
split_idx = int(len(encoded_text) * 0.9)
train_text = encoded_text[:split_idx]
val_text = encoded_text[split_idx:]

In [6]:
block_size = 64
def create_sequences(data, block_size):
    inputs = []
    targets = []

    for i in range(len(data) - block_size):
        input_seq = data[i:i + block_size]
        target_seq = data[i + 1:i + block_size + 1]

        inputs.append(input_seq)
        targets.append(target_seq)

    return inputs, targets
train_inputs, train_targets = create_sequences(train_text, block_size)
val_inputs, val_targets = create_sequences(val_text, block_size)


In [7]:
import numpy as np

def get_batch(inputs, targets, batch_size):
    assert len(inputs) == len(targets)
    indices = random.sample(range(len(inputs)), batch_size)
    input_batch = np.array([inputs[i] for i in indices], dtype=np.int32)
    target_batch = np.array([targets[i] for i in indices], dtype=np.int32)
    return input_batch, target_batch

In [8]:
import numpy as np

vocab_size = 65
embedding_dim = 128

embedding_matrix = np.random.randn(vocab_size, embedding_dim) * 0.01

def embedded_input(input_ids):
    return embedding_matrix[input_ids] + positional_embedding[np.arange(input_ids.shape[1])]

positional_embedding = np.random.randn(block_size, embedding_dim) * 0.01